In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_excel('./ENB2012_data.xlsx')

In [4]:
df.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area.1,Heating Load,Cooling Load
766,0.62,808.5,367.5,220.50,3.5,4,0.40,5,16.48,16.61
328,0.64,784.0,343.0,220.50,3.5,2,0.25,1,17.50,21.13
741,0.76,661.5,416.5,122.50,7.0,3,0.40,5,39.72,39.80
193,0.98,514.5,294.0,110.25,7.0,3,0.10,4,24.23,25.72
362,0.74,686.0,245.0,220.50,3.5,4,0.25,2,12.16,15.18


In [5]:
df.describe()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area.1,Heating Load,Cooling Load
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Relative Compactness  768 non-null    float64
 1   Surface Area          768 non-null    float64
 2   Wall Area             768 non-null    float64
 3   Roof Area             768 non-null    float64
 4   Overall Height        768 non-null    float64
 5   Orientation           768 non-null    int64  
 6   Glazing Area          768 non-null    float64
 7   Glazing Area.1        768 non-null    int64  
 8   Heating Load          768 non-null    float64
 9   Cooling Load          768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


#### Pisahin data test & train

In [7]:
x = df.drop(['Heating Load', 'Cooling Load'], axis=1)
yHeating = df[['Heating Load']]
yCooling = df[['Cooling Load']]

#### preparation untuk prediksi Heating Load & Cooling Load

In [8]:
from sklearn.model_selection import train_test_split

## Heating Load
xTrain, xTest, yTrain, yTest = train_test_split(x, yHeating, test_size =0.2, random_state = 9)

## Cooling Load
xTrain, xTest, yTrain2, yTest2 = train_test_split(x, yCooling, test_size=0.2, random_state= 9)

#### MOdeling 1

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(random_state=42)


#### Modeling  parameter default

##### prediksi Heating Load

In [10]:
## train model
yTrainmodelHeating = np.array(yTrain)
yTrainmodelHeating = yTrainmodelHeating.reshape(-1)
modelHeating1 = model.fit(xTrain, yTrainmodelHeating)

In [11]:
### prediksi heating load
ypredHeating1 = modelHeating1.predict(xTest)
mae = mean_absolute_error(yTest, ypredHeating1)
print(f'MAE : {mae}')

MAE : 0.3277811688311696


##### Prediksi Cooling Load

In [12]:
### train model

yTrainCooling = np.array(yTrain2)
yTrainCooling = yTrainCooling.reshape(-1)
modelCooling1 = model.fit(xTrain, yTrainCooling)

In [13]:
### prediksi cooling load
ypredCooling1 = modelCooling1.predict(xTest)
mae = mean_absolute_error(yTest2, ypredCooling1)
print(f'MAE : {mae}')

MAE : 1.0072727272727224


#### Model dengan parameter

In [14]:
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 5, 10, 20],
    # 'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

##### Prediksi Heating Load

In [15]:
grid_search.fit(xTrain, yTrainmodelHeating)
print(f'parameter terdabes : {grid_search.best_params_}')
print(f'\n score terdabes : {grid_search.best_score_}')
print(f'\n score terdabes : {grid_search.best_estimator_}')

parameter terdabes : {'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

 score terdabes : -0.24077494536425254

 score terdabes : RandomForestRegressor(max_features=None, random_state=42)


In [16]:
modelgue.fit(xTrain,yTrainmodelHeating)

NameError: name 'modelgue' is not defined

In [ ]:
modelgue = grid_search.best_estimator_
prediksi = modelgue.predict(xTest)
mae = mean_absolute_error(yTest, prediksi)
print(f'mae : {mae}')


mae : 0.3277811688311696


##### Prediksi Cooling Load    

In [ ]:
grid_search.fit(xTrain, yTrainCooling)
print(f'parameter terdabes : {grid_search.best_params_}')
print(f'\n score terdabes : {grid_search.best_score_}')
print(f'\n score terdabes : {grid_search.best_estimator_}')

parameter terdabes : {'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

 score terdabes : -2.618217933687841

 score terdabes : RandomForestRegressor(max_depth=10, max_features='sqrt', random_state=42)


In [ ]:
modelgue2 = grid_search.best_estimator_
prediksi2  = modelgue2.predict(xTest)
mae2 = mean_absolute_error(yTest, prediksi2)
print(f'mae : {mae2}')

mae : 2.6286758174658105


In [ ]:
prediksi2 = pd.DataFrame(prediksi2)
prediksi = pd.DataFrame(prediksi)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(yTest['Heating Load'], prediksi[:, 0], alpha=0.5, label='Heating Load')
plt.scatter(yTest['Cooling Load'], prediksi2[:, 1], alpha=0.5, label='Cooling Load', color='r')
plt.plot([0, 50], [0, 50], '--k')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.legend()
plt.title('Actual vs Predicted')
plt.show()

KeyError: (slice(None, None, None), 0)

<Figure size 1000x600 with 0 Axes>